In [1]:
%reset -f

In [2]:
import pandas as pd
from sssom.parsers import parse_sssom_table
from sssom import compare_dataframes
from sssom.parsers import split_dataframe
from sssom.util import MappingSetDataFrame
from os.path import join
from oaklib import OntologyResource
from oaklib.implementations import SqlImplementation
import textdistance

In [3]:
lexmatch_file = "../mappings/mondo-sources-all-lexical.sssom.tsv"
mondo_sssom = "../ontology/tmp/mondo.sssom.tsv"
dir_name = "dataframes"

In [4]:
# Functions

def add_distance(df, col_name, txt_dist_pkg):
    df\
    .insert(\
            len(df.columns),\
            col_name,\
            df.apply\
            (\
                lambda x: txt_dist_pkg\
                 (\
                    x.subject_label.lower(), x.object_label.lower()\
                    if pd.notnull(x.object_label) else "99"\
                 ),\
                axis=1,\
            )\
           )
    
def flip_predicate(predicate_id):
    flip_dict = {
        "skos:closeMatch": "skos:relatedMatch",
        "skos:relatedMatch": "skos:closeMatch",
        "skos:narrowMatch" : "skos:broadMatch",
        "skos:broadMatch" : "skos:narrowMatch",
        "skos:exactMatch" : "skos:exactMatch"
    }
    
    return flip_dict[predicate_id]

def compare_and_comment_df(mondo_df, lex_df):
    df = compare_dataframes(mondo_df, lex_df).combined_dataframe
    df['comment'] = df['comment'].str.replace('UNIQUE_1', "MONDO_MAPPINGS")
    df['comment'] = df['comment'].str.replace('UNIQUE_2', "LEXMATCH")
    return df

def get_unmapped_df(comparison_df):
    mappings = ["LEXMATCH", "MONDO_MAPPINGS"]
    unmapped_df = comparison_df[
        (comparison_df['comment'].str.contains("|".join(mappings)))
    ]

    return unmapped_df

def export_unmatched_exact(unmapped_df, match_type, fn):
    unmapped_exact = unmapped_df[(unmapped_df['comment'] == match_type) & (unmapped_df['predicate_id'] == 'skos:exactMatch')]
    unmapped_exact.to_csv(join(dir_name, fn), sep='\t', index = False)
    return unmapped_exact.head()

def make_msdf(comparison_df, prefix_map, meta):
    combined_msdf = MappingSetDataFrame(df=comparison_df, prefix_map=prefix_map, metadata=meta)
    return combined_msdf


In [5]:
%%time
msdf_lex = parse_sssom_table(lexmatch_file)
msdf_mondo = parse_sssom_table(mondo_sssom)

# Use OAK to get 'object_label'
ontology_resource = OntologyResource(slug='../ontology/tmp/merged.db', local=True)
oi = SqlImplementation(ontology_resource)

CPU times: user 3min 28s, sys: 5.06 s, total: 3min 33s
Wall time: 3min 35s


In [6]:
mondo_object_prefixes = msdf_mondo.df['object_id'].str.split(':').apply(lambda x: x[0] ).drop_duplicates()
mondo_subject_prefixes = msdf_mondo.df['subject_id'].str.split(':').apply(lambda x: x[0] ).drop_duplicates()
mondo_predicate_ids = msdf_mondo.df['predicate_id'].drop_duplicates()

print(f"mondo_subject_prefixes:\n {mondo_subject_prefixes} \n \
        lex_object_prefixes:\n {mondo_object_prefixes} \n \
        predicate_ids: \n {mondo_predicate_ids}")

mondo_subject_prefixes:
 0    MONDO
Name: subject_id, dtype: object 
         lex_object_prefixes:
 0           SCTID
1            MESH
2            DOID
3        Orphanet
4            NCIT
5            UMLS
11         OMIMPS
51        ICD10CM
172          OMIM
604        MedDRA
657      ICD10WHO
25506      MEDGEN
Name: object_id, dtype: object 
         predicate_ids: 
 0      skos:exactMatch
111    skos:broadMatch
Name: predicate_id, dtype: object


In [7]:
%%time
# msdf_mondo.df[msdf_mondo.df['object_id'].str.contains('ICD')]
# "ICD10CM", "MONDO"
# msdf_mondo.df = msdf_mondo.df[(condition_1 & condition_2) | (condition_3 & condition_4)]
# msdf_mondo.df['object_label'] = msdf_mondo.df['object_id'].apply(lambda x: oi.label(x))

msdf_mondo.df['object_label'] = msdf_mondo.df['object_id'].apply(lambda x: oi.label(x))
msdf_mondo.df.head()


CPU times: user 1min 21s, sys: 6.81 s, total: 1min 28s
Wall time: 1min 31s


,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label
0,MONDO:0000001,disease or disorder,skos:exactMatch,SCTID:64572001,Unspecified,None
1,MONDO:0000001,disease or disorder,skos:exactMatch,MESH:D004194,Unspecified,None
2,MONDO:0000001,disease or disorder,skos:exactMatch,DOID:4,Unspecified,disease
3,MONDO:0000001,disease or disorder,skos:exactMatch,Orphanet:377788,Unspecified,Disease
4,MONDO:0000001,disease or disorder,skos:exactMatch,NCIT:C2991,Unspecified,Disease or Disorder


In [8]:
condition_1 = msdf_mondo.df['subject_id'].str.contains("MONDO")
condition_2 = msdf_mondo.df['object_id'].str.contains("ICD10CM")
condition_3 = msdf_mondo.df['object_id'].str.contains('|'.join((["OMIM","OMIMPS"])))
condition_4 = msdf_mondo.df['object_id'].str.contains("Orphanet")
condition_5 = msdf_mondo.df['object_id'].str.contains("DOID")

mondo_icd_df = msdf_mondo.df[condition_1 & condition_2]
mondo_omim_df = msdf_mondo.df[condition_1 & condition_3]
mondo_ordo_df = msdf_mondo.df[condition_1 & condition_4]
mondo_doid_df = msdf_mondo.df[condition_1 & condition_5]

mondo_icd_df.head()
mondo_omim_df.head()
mondo_ordo_df.head()
# mondo_doid_df.head()


,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label
3,MONDO:0000001,disease or disorder,skos:exactMatch,Orphanet:377788,Unspecified,Disease
20,MONDO:0000023,infantile liver failure,skos:exactMatch,Orphanet:464724,Unspecified,Fever-associated acute infantile liver failure...
26,MONDO:0000044,hereditary hypophosphatemic rickets,skos:exactMatch,Orphanet:437,Unspecified,Hypophosphatemic rickets
33,MONDO:0000050,isolated congenital growth hormone deficiency,skos:exactMatch,Orphanet:631,Unspecified,Non-acquired isolated growth hormone deficiency
49,MONDO:0000087,polymicrogyria,skos:exactMatch,Orphanet:35981,Unspecified,Polymicrogyria


In [9]:
lex_object_prefixes = msdf_lex.df['object_id'].str.split(':').apply(lambda x: x[0] ).drop_duplicates()
lex_subject_prefixes = msdf_lex.df['subject_id'].str.split(':').apply(lambda x: x[0] ).drop_duplicates()
lex_predicate_ids = msdf_lex.df['predicate_id'].drop_duplicates()

print(f"subject_prefixes:\n {lex_subject_prefixes} \n \
      object_prefixes:\n {lex_object_prefixes} \n \
      predicat_ids:\n {lex_predicate_ids} ")


subject_prefixes:
 0             DOID
151307     ICD10CM
166651    ICD10WHO
169860       MONDO
425468        NCIT
458795        OMIM
474649    Orphanet
Name: subject_id, dtype: object 
       object_prefixes:
 0         MONDO
7          NCIT
8      Orphanet
11     ICD10WHO
20         DOID
37      ICD10CM
199        OMIM
Name: object_id, dtype: object 
       predicat_ids:
 0       skos:broadMatch
1       skos:exactMatch
18      skos:closeMatch
176    skos:narrowMatch
Name: predicate_id, dtype: object 


### Flipping subject_id and object_id if MONDO not subject_id prefix

**Predicate impact**

- skos:closeMatch <=> skos:relatedMatch
- skos:narrowMatch < => skos:broadMatch

In [10]:
condition_1 = msdf_lex.df['subject_id'].str.contains("MONDO")
condition_2 = msdf_lex.df['object_id'].str.contains("ICD10CM")
condition_3 = msdf_lex.df['object_id'].str.contains('|'.join((["OMIM","OMIMPS"])))
condition_4 = msdf_lex.df['object_id'].str.contains('|'.join((["ORDO","Orphanet"])))
condition_5 = msdf_lex.df['object_id'].str.contains("DOID")
condition_mondo_obj = msdf_lex.df['object_id'].str.contains("MONDO")
non_mondo_subjects_df =  pd.DataFrame(msdf_lex.df[(~condition_1 & condition_mondo_obj)])
mondo_subjects_df = pd.DataFrame(msdf_lex.df[(condition_1 & ~condition_mondo_obj)])
print(len(mondo_subjects_df))
non_mondo_subjects_df.head()


44371


,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,mapping_tool,confidence,subject_match_field,object_match_field,match_string
0,DOID:0001816,angiosarcoma,skos:broadMatch,MONDO:0003022,pediatric angiosarcoma,semapv:LexicalMatching,oaklib,0.800000,rdfs:label,oio:hasBroadSynonym,angiosarcoma
1,DOID:0001816,angiosarcoma,skos:exactMatch,MONDO:0016982,angiosarcoma,semapv:LexicalMatching,oaklib,0.941176,oio:hasDbXref,oio:hasDbXref,mesh:d006394
2,DOID:0001816,angiosarcoma,skos:exactMatch,MONDO:0016982,angiosarcoma,semapv:LexicalMatching,oaklib,0.800000,rdfs:label,oio:hasExactSynonym,angiosarcoma
3,DOID:0001816,angiosarcoma,skos:exactMatch,MONDO:0016982,angiosarcoma,semapv:RegularExpressionReplacement,oaklib,0.800000,rdfs:label,oio:hasExactSynonym,angiosarcoma
4,DOID:0001816,angiosarcoma,skos:exactMatch,MONDO:0016982,angiosarcoma,semapv:LexicalMatching,oaklib,0.800000,oio:hasExactSynonym,oio:hasExactSynonym,hemangiosarcoma


In [11]:
desired_sequence = ['subject_id', 'subject_label', 'predicate_id', 'object_id',
       'object_label', 'mapping_justification', 'mapping_tool', 'confidence',
       'subject_match_field', 'object_match_field', 'match_string']

new_subjects_df = non_mondo_subjects_df.rename(columns={
    'subject_id':'object_id', 
    'subject_label':'object_label',
    'object_id':'subject_id', 
    'object_label':'subject_label'
    
})


new_subjects_df = new_subjects_df[desired_sequence]
new_subjects_df["predicate_id"] = new_subjects_df["predicate_id"].apply(lambda x: flip_predicate(x))
print(len(new_subjects_df))
new_subjects_df.head()


103462


,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,mapping_tool,confidence,subject_match_field,object_match_field,match_string
0,MONDO:0003022,pediatric angiosarcoma,skos:narrowMatch,DOID:0001816,angiosarcoma,semapv:LexicalMatching,oaklib,0.800000,rdfs:label,oio:hasBroadSynonym,angiosarcoma
1,MONDO:0016982,angiosarcoma,skos:exactMatch,DOID:0001816,angiosarcoma,semapv:LexicalMatching,oaklib,0.941176,oio:hasDbXref,oio:hasDbXref,mesh:d006394
2,MONDO:0016982,angiosarcoma,skos:exactMatch,DOID:0001816,angiosarcoma,semapv:LexicalMatching,oaklib,0.800000,rdfs:label,oio:hasExactSynonym,angiosarcoma
3,MONDO:0016982,angiosarcoma,skos:exactMatch,DOID:0001816,angiosarcoma,semapv:RegularExpressionReplacement,oaklib,0.800000,rdfs:label,oio:hasExactSynonym,angiosarcoma
4,MONDO:0016982,angiosarcoma,skos:exactMatch,DOID:0001816,angiosarcoma,semapv:LexicalMatching,oaklib,0.800000,oio:hasExactSynonym,oio:hasExactSynonym,hemangiosarcoma


### Combine dfs where subjec_id prefix is MONDO

In [12]:
lex_df = pd.concat([mondo_subjects_df,new_subjects_df], ignore_index=True)
print(len(lex_df))

lex_df.head()

147833


,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,mapping_tool,confidence,subject_match_field,object_match_field,match_string
0,MONDO:0000001,disease or disorder,skos:exactMatch,Orphanet:557493,disorder,semapv:LexicalMatching,oaklib,0.8,oio:hasExactSynonym,rdfs:label,disorder
1,MONDO:0000001,disease or disorder,skos:exactMatch,NCIT:C25457,Condition,semapv:LexicalMatching,oaklib,0.8,oio:hasExactSynonym,rdfs:label,condition
2,MONDO:0000001,disease or disorder,skos:exactMatch,NCIT:C156809,Medical Condition,semapv:LexicalMatching,oaklib,0.8,oio:hasExactSynonym,rdfs:label,medical condition
3,MONDO:0000004,adrenocortical insufficiency,skos:exactMatch,NCIT:C113211,Hypocortisolemia,semapv:LexicalMatching,oaklib,0.8,oio:hasExactSynonym,rdfs:label,hypocortisolemia
4,MONDO:0000022,nocturnal enuresis,skos:exactMatch,OMIM:600631,"enuresis, nocturnal, 1",semapv:LexicalMatching,oaklib,0.8,oio:hasExactSynonym,oio:hasExactSynonym,bedwetting


In [13]:
# msdf_lex.df[msdf_lex.df['object_id'].str.contains('ICD')]
condition_1 = lex_df['subject_id'].str.contains("MONDO")
condition_2 = lex_df['object_id'].str.contains("ICD10CM")
condition_3 = lex_df['object_id'].str.contains('|'.join((["OMIM","OMIMPS"])))
condition_4 = lex_df['object_id'].str.contains("Orphanet")
condition_5 = lex_df['object_id'].str.contains("DOID")


mondo_icd_lex_df = lex_df[(condition_1 & condition_2)]
mondo_omim_lex_df = lex_df[(condition_1 & condition_3)]
mondo_ordo_lex_df = lex_df[(condition_1 & condition_4)]
mondo_doid_lex_df = lex_df[(condition_1 & condition_5)]

mondo_icd_lex_df.head()
mondo_omim_lex_df.head() # NO ROWS
mondo_ordo_lex_df.head() # NO ROWS
mondo_doid_lex_df.head()


,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,mapping_tool,confidence,subject_match_field,object_match_field,match_string
968,MONDO:0004990,breast tumor luminal A or B,skos:narrowMatch,DOID:0060548,luminal breast carcinoma A,semapv:LexicalMatching,oaklib,0.800000,oio:hasExactSynonym,oio:hasBroadSynonym,luminal breast cancer
969,MONDO:0004990,breast tumor luminal A or B,skos:narrowMatch,DOID:0060548,luminal breast carcinoma A,semapv:LexicalMatching,oaklib,0.800000,oio:hasExactSynonym,oio:hasBroadSynonym,breast tumor luminal
44371,MONDO:0003022,pediatric angiosarcoma,skos:narrowMatch,DOID:0001816,angiosarcoma,semapv:LexicalMatching,oaklib,0.800000,rdfs:label,oio:hasBroadSynonym,angiosarcoma
44372,MONDO:0016982,angiosarcoma,skos:exactMatch,DOID:0001816,angiosarcoma,semapv:LexicalMatching,oaklib,0.941176,oio:hasDbXref,oio:hasDbXref,mesh:d006394
44373,MONDO:0016982,angiosarcoma,skos:exactMatch,DOID:0001816,angiosarcoma,semapv:LexicalMatching,oaklib,0.800000,rdfs:label,oio:hasExactSynonym,angiosarcoma


In [14]:
%%time
# comparison_ms_diff = compare_dataframes(msdf_mondo.df, msdf_lex.df)
# comparison_df = comparison_ms_diff.combined_dataframe
# comparison_df['comment'] = comparison_df['comment'].str.replace('UNIQUE_1', "MONDO_MAPPINGS")
# comparison_df['comment'] = comparison_df['comment'].str.replace('UNIQUE_2', "LEXMATCH")
# comparison_df.head()

# Comparisons
icd_comparison_df = compare_and_comment_df(mondo_icd_df, mondo_icd_lex_df)
omim_comparison_df = compare_and_comment_df(mondo_omim_df, mondo_omim_lex_df)
ordo_comparison_df = compare_and_comment_df(mondo_ordo_df, mondo_ordo_lex_df)
doid_comparison_df = compare_and_comment_df(mondo_doid_df, mondo_doid_lex_df)


CPU times: user 1min 23s, sys: 314 ms, total: 1min 24s
Wall time: 1min 24s


In [15]:
icd_comparison_df
omim_comparison_df
ordo_comparison_df
doid_comparison_df

,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
3000,MONDO:0001362,obsolete leukocoria,skos:exactMatch,DOID:11772,Unspecified,leukocoria,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
15976,MONDO:0005525,T-cell leukemia,skos:exactMatch,DOID:715,Unspecified,obsolete T-cell lymphoblastic leukemia/lymphoma,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
1244,MONDO:0000789,obsolete Atlantic cod allergy,skos:exactMatch,DOID:0060514,Unspecified,Atlantic cod allergy,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
12253,MONDO:0004391,obsolete adult extraosseous chondrosarcoma,skos:exactMatch,DOID:7902,Unspecified,adult extraosseous chondrosarcoma,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
879,MONDO:0000590,autoimmune disorder of peripheral nervous system,skos:exactMatch,DOID:0060033,Unspecified,autoimmune disease of peripheral nervous system,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
73522,MONDO:0008913,"cardiac valvular defect, developmental",skos:exactMatch,DOID:0080633,semapv:LexicalMatching,developmental cardiac valvular defect,COMMON_TO_BOTH,oaklib,0.941176,oio:hasDbXref,oio:hasDbXref,omim:212093
13820,MONDO:0004868,biliary tract disorder,skos:exactMatch,DOID:9741,Unspecified,biliary tract disease,COMMON_TO_BOTH,NaN,NaN,NaN,NaN,NaN
140342,MONDO:0004868,biliary tract disorder,skos:exactMatch,DOID:9741,semapv:LexicalMatching,biliary tract disease,COMMON_TO_BOTH,oaklib,0.941176,oio:hasDbXref,oio:hasDbXref,mesh:d001660
10934,MONDO:0003939,muscle tissue disorder,skos:exactMatch,DOID:66,Unspecified,muscle tissue disease,COMMON_TO_BOTH,NaN,NaN,NaN,NaN,NaN


In [16]:
icd_comparison_df['comment'].drop_duplicates()

18076     MONDO_MAPPINGS
141968          LEXMATCH
5244      COMMON_TO_BOTH
Name: comment, dtype: object

### Split into unmapped dataframes

In [17]:
unmapped_icd_df = get_unmapped_df(icd_comparison_df)
unmapped_omim_df = get_unmapped_df(omim_comparison_df)
unmapped_ordo_df = get_unmapped_df(ordo_comparison_df)
unmapped_doid_df = get_unmapped_df(doid_comparison_df)

unmapped_icd_df.head()


,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
18076,MONDO:0006133,cervical adenoid cystic carcinoma,skos:broadMatch,ICD10CM:C53.8,Unspecified,Malignant neoplasm of overlapping sites of cer...,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
63752,MONDO:0020109,obsolete constitutional megaloblastic anemia d...,skos:broadMatch,ICD10CM:D51.9,Unspecified,"Vitamin B12 deficiency anemia, unspecified",MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
16346,MONDO:0005645,ancylostomiasis,skos:broadMatch,ICD10CM:B76.9,Unspecified,"Hookworm disease, unspecified",MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
53821,MONDO:0016419,hereditary breast carcinoma,skos:broadMatch,ICD10CM:C50.8,Unspecified,Malignant neoplasm of overlapping sites of breast,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
19596,MONDO:0006687,burning mouth syndrome,skos:broadMatch,ICD10CM:K14.6,Unspecified,Glossodynia,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN


In [18]:
# Let reviewers check if this makes sense or no.
export_unmatched_exact(unmapped_icd_df, "LEXMATCH", "unmapped_icd_lex.tsv")
export_unmatched_exact(unmapped_omim_df, "LEXMATCH", "unmapped_omim_lex.tsv")
export_unmatched_exact(unmapped_ordo_df, "LEXMATCH", "unmapped_ordo_lex.tsv")
export_unmatched_exact(unmapped_doid_df, "LEXMATCH", "unmapped_doid_lex.tsv")


,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
54852,MONDO:0012621,deafness-infertility syndrome,skos:exactMatch,DOID:0060417,semapv:LexicalMatching,3p deletion syndrome,LEXMATCH,oaklib,0.941176,oio:hasDbXref,oio:hasDbXref,icd10cm:q93.5
118626,MONDO:0006948,retinal artery occlusion,skos:exactMatch,DOID:13094,semapv:LexicalMatching,branch retinal artery occlusion,LEXMATCH,oaklib,0.941176,oio:hasDbXref,oio:hasDbXref,mesh:d015356
57702,MONDO:0009124,Dubowitz syndrome,skos:exactMatch,DOID:0060581,semapv:LexicalMatching,Noonan syndrome 3,LEXMATCH,oaklib,0.941176,oio:hasDbXref,oio:hasDbXref,icd10cm:q87.1
81078,MONDO:0011113,Charcot-Marie-Tooth disease type 4C,skos:exactMatch,DOID:0110167,semapv:LexicalMatching,Charcot-Marie-Tooth disease axonal type 2K,LEXMATCH,oaklib,0.941176,oio:hasDbXref,oio:hasDbXref,icd10cm:g60.0
102031,MONDO:0014015,hereditary spastic paraplegia 56,skos:exactMatch,DOID:0110811,semapv:LexicalMatching,hereditary spastic paraplegia 6,LEXMATCH,oaklib,0.941176,oio:hasDbXref,oio:hasDbXref,icd10cm:g11.4


In [19]:
# Inspect why these are missing from SSSOM mappings
export_unmatched_exact(unmapped_icd_df, "MONDO_MAPPINGS", "unmapped_icd_mondo.tsv")
export_unmatched_exact(unmapped_omim_df, "MONDO_MAPPINGS", "unmapped_omim_mondo.tsv")
export_unmatched_exact(unmapped_ordo_df, "MONDO_MAPPINGS", "unmapped_ordo_mondo.tsv")
export_unmatched_exact(unmapped_doid_df, "MONDO_MAPPINGS", "unmapped_doid_mondo.tsv")

,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
3000,MONDO:0001362,obsolete leukocoria,skos:exactMatch,DOID:11772,Unspecified,leukocoria,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
15976,MONDO:0005525,T-cell leukemia,skos:exactMatch,DOID:715,Unspecified,obsolete T-cell lymphoblastic leukemia/lymphoma,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
1244,MONDO:0000789,obsolete Atlantic cod allergy,skos:exactMatch,DOID:0060514,Unspecified,Atlantic cod allergy,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
12253,MONDO:0004391,obsolete adult extraosseous chondrosarcoma,skos:exactMatch,DOID:7902,Unspecified,adult extraosseous chondrosarcoma,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
879,MONDO:0000590,autoimmune disorder of peripheral nervous system,skos:exactMatch,DOID:0060033,Unspecified,autoimmune disease of peripheral nervous system,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN


In [20]:
# # Add distances
# # Add Levenshtein distance [ 0: Perfect match]
# add_distance(unmapped_mondo_exact, "levenshtein_dist", textdistance.levenshtein.distance)
# # Add Jaccard Index [ 0: Perfect match]
# add_distance(unmapped_mondo_exact, "jaccard_index", textdistance.jaccard.distance)
# # Add Monge-Elkan Distance [ 0: Perfect match]
# add_distance(unmapped_mondo_exact, "monge_elkan", textdistance.monge_elkan.distance)
# unmapped_mondo_exact.to_csv(join(dir_name, "unmapped_mondo_exact.tsv"), sep='\t', index = False)
# unmapped_mondo_exact.head()


In [21]:
combined_msdf_icd = make_msdf(icd_comparison_df, msdf_lex.prefix_map, msdf_lex.metadata)
combined_msdf_omim = make_msdf(omim_comparison_df, msdf_lex.prefix_map, msdf_lex.metadata)
combined_msdf_ordo = make_msdf(ordo_comparison_df, msdf_lex.prefix_map, msdf_lex.metadata)
combined_msdf_doid = make_msdf(doid_comparison_df, msdf_lex.prefix_map, msdf_lex.metadata)

combined_msdf_icd.df.head()

,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
18076,MONDO:0006133,cervical adenoid cystic carcinoma,skos:broadMatch,ICD10CM:C53.8,Unspecified,Malignant neoplasm of overlapping sites of cer...,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
63752,MONDO:0020109,obsolete constitutional megaloblastic anemia d...,skos:broadMatch,ICD10CM:D51.9,Unspecified,"Vitamin B12 deficiency anemia, unspecified",MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
16346,MONDO:0005645,ancylostomiasis,skos:broadMatch,ICD10CM:B76.9,Unspecified,"Hookworm disease, unspecified",MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
53821,MONDO:0016419,hereditary breast carcinoma,skos:broadMatch,ICD10CM:C50.8,Unspecified,Malignant neoplasm of overlapping sites of breast,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
19596,MONDO:0006687,burning mouth syndrome,skos:broadMatch,ICD10CM:K14.6,Unspecified,Glossodynia,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN


In [22]:
# %%time
# df_dict = split_dataframe(combined_msdf)

In [23]:
# mondo_icd_list = [x for x in list(df_dict.keys()) if 'mondo' in x and "icd10" in x]
# mondo_icd_list

In [24]:
# for match in mondo_icd_list:
#     fn = match + ".tsv"
#     df_dict[match].df.to_csv(join(dir_name,fn), sep='\t', index = False)

In [25]:
# df_dict['mondo_exactmatch_icd10cm'].df